In [1]:
#Import libraries for simulation
import tensorflow as tf
import numpy as np

In [2]:
#Imports for visualization
import PIL.Image
from io import BytesIO
from IPython.display import Image, display

In [8]:
# build computational graph
node1 = tf.constant(3.0, tf.float32)
node2 = tf.constant(4.0)
print(node1, node2)

(<tf.Tensor 'Const_3:0' shape=() dtype=float32>, <tf.Tensor 'Const_4:0' shape=() dtype=float32>)


In [9]:
# create Session
sess = tf.Session()
print(sess.run([node1, node2]))

[3.0, 4.0]


In [15]:
# add two constant nodes and produce a new graph as follows
node3 = tf.add(node1, node2)
print"node3 : {}".format(node3)
print"sess.run(node3) : {}".format(sess.run(node3))

node3 : Tensor("Add_5:0", shape=(), dtype=float32)
sess.run(node3) : 7.0


In [16]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b  # + provides a shortcut for tf.add(a,b)

In [18]:
print sess.run(adder_node, {a:3, b:4.5})
print sess.run(adder_node, {a:[1,3], b:[2, 4]})

7.5
[ 3.  7.]


In [19]:
add_and_triple = adder_node * 3.
print(sess.run(add_and_triple, {a: 3, b:4.5}))

22.5


In [20]:
W = tf.Variable([.3], tf.float32)
b = tf.Variable([-.3], tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W * x + b

In [24]:
init = tf.global_variables_initializer()
sess.run(init)

In [26]:
print init

name: "init"
op: "NoOp"
input: "^Variable/Assign"
input: "^Variable_1/Assign"



In [27]:
print(sess.run(linear_model, {x:[1,2,3,4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


In [29]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)
print(sess.run(loss, {x:[1,2,3,4], y:[0,-1,-2,-3]}))

23.66


In [30]:
fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb])
print(sess.run(loss, {x:[1,2,3,4], y:[0,-1,-2,-3]}))

0.0


In [32]:
import tensorflow as tf

# 변수를 0으로 초기화
state = tf.Variable(0, name="counter")

# state에 1을 더할 오퍼레이션 생성
one = tf.constant(1)
new_value = tf.add(state, one)
update = tf.assign(state, new_value)

# 그래프는 처음에 변수를 초기화해야 합니다. 아래 함수를 통해 init 오퍼레이션을 만듭니다.   
init_op = tf.global_variables_initializer()

# 그래프를 띄우고 오퍼레이션들을 실행
with tf.Session() as sess:
# 초기화 오퍼레이션 실행
  sess.run(init_op)
# state의 초기 값을 출력
  print(sess.run(state))
# state를 갱신하는 오퍼레이션을 실행하고, state를 출력
  for _ in range(3):
    sess.run(update)
    print(sess.run(state))

0
1
2
3


In [ ]:
"""A very simple MNIST classifier.
See extensive documentation at
http://tensorflow.org/tutorials/mnist/beginners/index.md
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys

# Import data
from tensorflow.examples.tutorials.mnist import input_data

import tensorflow as tf

FLAGS = None



# Load data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
#   mnist = input_data.read_data_sets(FLAGS.data_dir, one_hot=True)

# Create the model
x = tf.placeholder(tf.float32, [None, 784])
# placeholder: a value that we'll input
# [None,784] => any length of dimension, 784-dimensional vector / 2D tensor of floating point numbers
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
# weights and biases
# W => [784,10] => 784-dimensional image vectors, 10-dimensional vectors of evidence
# y = tf.nn.softmax(tf.matmul(x, W) + b)
y = tf.matmul(x, W) + b


# Define loss and optimizer
y_ = tf.placeholder(tf.float32, [None, 10])

# The raw formulation of cross-entropy,
#
#   tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(tf.nn.softmax(y)),
#                                 reduction_indices=[1]))
#
# can be numerically unstable.
#
# So here we use tf.nn.softmax_cross_entropy_with_logits on the raw
# outputs of 'y', and then average across the batch.
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y, y_))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
# minimize cross_entropy using the gradient descent algorithm
# learning rate: 0.5

# initialize the variables we created
# init = tf.global_variables_initializer()

sess = tf.InteractiveSession()
# Train
tf.initialize_all_variables().run()
# tf.global_variables_initializer().run()
for _ in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
# Run training step 1,000 times


# Test trained model
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
# tf.argmax: gives the index of the highest entry in a tensor along some axis
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# cast to floating point numbers => take mean
# [T,F,T,T] => [1,0,1,1] => 0.75
print(sess.run(accuracy, feed_dict={x: mnist.test.images,y_: mnist.test.labels}))

#     if __name__ == '__main__':
#         parser = argparse.ArgumentParser()
#         parser.add_argument('--data_dir', type=str, default='/tmp/tensorflow/mnist/input_data',
#                           help='Directory for storing input data')
#         FLAGS, unparsed = parser.parse_known_args()
#         tf.app.run(main=main, argv=[sys.argv[0]] + unparsed)